In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from IPython.display import clear_output
from collections import Counter
from functools import reduce
import itertools
from pbpstats.client import Client
import matplotlib.pyplot as plt
import seaborn as sns
from nbafuns import *

In [2]:
settings = {
    "Games": {"source": "web", "data_provider": "data_nba"},
     "dir": "G:/My Drive/Sra_Coding/NBA/pbpdata",
}
client = Client(settings)
# ID of all games for 2020-21 Season
season = client.Season("wnba", "2021", "Regular Season")
games_id = []
k = 0
for final_game in season.games.final_games:
    k += 1
    clear_output(wait=True)
    print(k)
    games_id.append(final_game['game_id'])

74


In [15]:
settings = {
    "Boxscore": {"source": "file", "data_provider": "data_nba"},
    "Possessions": {"source": "file", "data_provider": "data_nba"},
    "dir": "G:/My Drive/Sra_Coding/NBA/pbpdata"
}
client = Client(settings)
games_list = []
bad_games_list = []
k = 0
for gameid in games_id:
    clear_output(wait=True)
    k += 1
    print(k)
    try:
        games_list.append(client.Game(gameid))
    except:
        print(gameid)
        bad_games_list.append(gameid)
        continue

74


In [16]:
len(bad_games_list)

1

In [17]:
player_dict = get_players(league='WNBA',from_year=2021, to_year=2021)
player_id = [player['pID'] for player in player_dict]
team_dict = get_teams(league='WNBA') # Creating Team Dictionary
team_id = [x['TeamID'] for x in team_dict ]

In [18]:
game = games_list[0]
possession = game.possessions.items[2]
possession_event = possession.events
event = possession_event[0]

In [24]:
# Get Location Data
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import Rebound
game_id, period, clock, seconds_remaining,poss_length = [],[],[],[],[]
team_id, player1_id= [],[]
locX, locY, distance, shot_value, shot_type = [],[],[],[],[]
is_made, is_putback, is_assisted, player2_id  = [],[],[],[]
len_diffp = 0
pos_store = []
i = 0
ii = 0
for game in games_list:
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if isinstance(possession_event, FieldGoal):
                try: 
                    game_id.append(possession_event.game_id)
                    period.append(possession_event.period)
                    clock.append(possession_event.clock)
                    seconds_remaining.append(possession_event.seconds_remaining)
                    poss_length.append(possession_event.seconds_since_previous_event)
                    team_id.append(possession_event.team_id)
                    player1_id.append(possession_event.player1_id)
                    locX.append(possession_event.locX)
                    locY.append(possession_event.locY)
                    distance.append(possession_event.distance)
                    shot_value.append(possession_event.shot_value)
                    shot_type.append(possession_event.shot_type)
                    is_made.append(possession_event.is_made)
                    
                    is_assisted.append(possession_event.is_assisted)
                    if possession_event.is_assisted:
                        player2_id.append(possession_event.player2_id)
                    else:
                        player2_id.append(0)
                    ii += 1
                    # print("ii = {0}".format(ii))
                    is_putback.append(possession_event.is_putback)
                except:
                    is_putback.append(False)
    i += 1
    print("i = {0}".format(i))
    clear_output(wait=True)

i = 73


In [25]:
player1_name = []
for pID in player1_id:
    player1_name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
player1_name = list(itertools.chain(*player1_name))
player2_name = []
for pID in player2_id:
    if pID==0:
        player2_name.append(['None'])
    else:     
        player2_name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
player2_name = list(itertools.chain(*player2_name))
team_name = []
for tID in team_id:  
    team_name.append([team['Team'] for team in team_dict if team['TeamID'] == tID])
team_name = list(itertools.chain(*team_name))

In [26]:
data = pd.DataFrame({'game_id':game_id,'period':period,'clock':clock,'seconds_remaining':seconds_remaining,'poss_length':poss_length,'team_id':team_id,'team_name':team_name,'player_id':player1_id,'player_name':player1_name,'locX': locX,'locY':locY, 'distance':distance,'shot_value':shot_value,'shot_type':shot_type,'is_made':is_made,'is_assisted':is_assisted,'player_ast_id':player2_id,'player_ast_name':player2_name,'is_putback':is_putback})
# data = data.sort_values(by=['Assists'],ascending=False)
# data = data.reset_index(drop=True)
data.head()

,game_id,period,clock,seconds_remaining,poss_length,team_id,team_name,player_id,player_name,locX,locY,distance,shot_value,shot_type,is_made,is_assisted,player_ast_id,player_ast_name,is_putback
0,1022100001,1,09:37,577.0,23.0,1611661325,Indiana Fever,202658,Danielle Robinson,18,16,2.4,2,AtRim,False,False,0,None,False
1,1022100001,1,09:16,556.0,7.0,1611661313,New York Liberty,1629477,Sabrina Ionescu,120,129,17.6,2,LongMidRange,False,False,0,None,False
2,1022100001,1,08:47,527.0,17.0,1611661313,New York Liberty,1630446,Michaela Onyenwere,160,198,25.5,3,Arc3,True,True,1628244,Sami Whitcomb,False
3,1022100001,1,08:23,503.0,24.0,1611661325,Indiana Fever,1628909,Kelsey Mitchell,101,17,10.2,2,ShortMidRange,False,False,0,None,False
4,1022100001,1,08:18,498.0,3.0,1611661325,Indiana Fever,1628927,Victoria Vivians,24,251,25.2,3,Arc3,False,False,0,None,False


In [27]:
data.to_csv ('data\WNBA_Shot_Loc_2021.csv',index=False)